In [1]:
import os
from sklearn.datasets import fetch_mldata
import random
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torch.nn as nn
import torch.nn.parallel
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import cv2
import numpy as np

torch.cuda.set_device(1)

In [2]:
import csv
dicti = {}
for line in open("/home/hubo/notebook/data/ISBI2016_ISIC_Part3_Training_GroundTruth.csv"):
    title, label = line.split(",")
    if label[0] == 'b':
        dicti[title] = 0
    elif label[0] == 'm':
        dicti[title] = 1
        
dicti

{'ISIC_0000000': 0,
 'ISIC_0000001': 0,
 'ISIC_0000002': 1,
 'ISIC_0000004': 1,
 'ISIC_0000006': 0,
 'ISIC_0000007': 0,
 'ISIC_0000008': 0,
 'ISIC_0000009': 0,
 'ISIC_0000010': 0,
 'ISIC_0000011': 0,
 'ISIC_0000016': 0,
 'ISIC_0000017': 0,
 'ISIC_0000018': 0,
 'ISIC_0000019': 0,
 'ISIC_0000021': 0,
 'ISIC_0000024': 0,
 'ISIC_0000025': 0,
 'ISIC_0000026': 1,
 'ISIC_0000028': 0,
 'ISIC_0000029': 1,
 'ISIC_0000030': 1,
 'ISIC_0000031': 1,
 'ISIC_0000032': 0,
 'ISIC_0000034': 0,
 'ISIC_0000035': 1,
 'ISIC_0000038': 0,
 'ISIC_0000039': 0,
 'ISIC_0000041': 0,
 'ISIC_0000042': 0,
 'ISIC_0000044': 0,
 'ISIC_0000045': 0,
 'ISIC_0000046': 1,
 'ISIC_0000047': 0,
 'ISIC_0000048': 0,
 'ISIC_0000049': 1,
 'ISIC_0000050': 0,
 'ISIC_0000051': 0,
 'ISIC_0000054': 1,
 'ISIC_0000055': 0,
 'ISIC_0000058': 0,
 'ISIC_0000059': 0,
 'ISIC_0000060': 0,
 'ISIC_0000061': 0,
 'ISIC_0000062': 0,
 'ISIC_0000063': 0,
 'ISIC_0000065': 0,
 'ISIC_0000067': 0,
 'ISIC_0000068': 0,
 'ISIC_0000073': 0,
 'ISIC_0000074': 1,


In [3]:
def dataset_img_resize():
    X_train = np.zeros(0)
    X_label = np.zeros(0)
    root_dir = '/home/hubo/notebook/data/skin lesion/'
    imageList = os.listdir(root_dir)
    m = 0
    
    for n in imageList:
        m = n[0:12]
        print (m)
        label = np.asarray(dicti.get(m),dtype=int)
        n = root_dir + n
        img = cv2.imread(n, 3)
        hight, width = img.shape[0], img.shape[1]
        
        resize = cv2.resize(img[hight/2-300:hight/2+300,width/2-300:width/2+300,:],(256,256))
        resize = resize[:,:,::-1].transpose((2,0,1))
        resize = resize.astype(np.float32)/(255.0/2) - 1.0
        resize = resize.reshape((1, 3, 256, 256))

        if X_train.shape[0] == 0:
            X_label = label.reshape((1,1)).astype(int)
            X_train = resize.reshape((1,1,3,256,256)).astype(np.float32)
        else:
            X_label = np.concatenate((X_label,label.reshape((1,1)).astype(int)),axis=0)
            X_train = np.concatenate((X_train,resize.reshape((1,1,3,256,256))),axis=0)
                    #print (X_train.shape)
            
   # np.random.seed(1234) # set seed for deterministic ordering
  #  p = np.random.permutation(X_train.shape[0])
  #  X_train = X_train[p]
    return X_train,X_label

#X_data,X_label = dataset_img_resize()
#X_data.shape

#np.save('skinlesion256256.npy',X_data)

X_data,X_label = np.load('skinlesion256256.npy'),np.load('skinlesion256256_label.npy')

In [4]:
X_train, X_test = X_data[0:640], X_data[616:680]
X_train_label, X_test_label = X_label[0:640], X_label[616:680]
X_train.resize(10,64,3,256,256)
X_test.resize(1,64,3,256,256)
X_train_label.resize(10,64)
X_test_label.resize(1,64)

X_test_label.shape

(1, 64)

In [5]:
import torchvision
model = torchvision.models.resnet18(pretrained=True)
model.cuda()

m = nn.Linear(1000, 2)
m.cuda()

#optimizer = optim.Adam([
#                {'params': m.parameters()},            
#                {'params': model.parameters()}
#            ], 0.00005)

optimizer = torch.optim.SGD(model.parameters(), 0.1,
                                momentum=0.9,
                                weight_decay=1e-4)

criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()

In [6]:
X_train = Variable(torch.FloatTensor(X_train))
X_train_label = Variable(torch.LongTensor(X_train_label))
train_loader = torch.utils.data.TensorDataset(X_train,X_train_label)
train_iter = iter(train_loader)

X_test = Variable(torch.FloatTensor(X_test))
X_test_label = Variable(torch.LongTensor(X_test_label))
test_loader = torch.utils.data.TensorDataset(X_test,X_test_label)
test_iter = iter(test_loader)


In [7]:
import math
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        return torch.cat([x, out], 1)

class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BottleneckBlock, self).__init__()
        inter_planes = out_planes * 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.conv2 = nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        out = self.conv2(self.relu(self.bn2(out)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return torch.cat([x, out], 1)

class TransitionBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(TransitionBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return F.avg_pool2d(out, 2)

class DenseBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, growth_rate, block, dropRate=0.0):
        super(DenseBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, growth_rate, nb_layers, dropRate)
    def _make_layer(self, block, in_planes, growth_rate, nb_layers, dropRate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(in_planes+i*growth_rate, growth_rate, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class DenseNet3(nn.Module):
    def __init__(self, depth, num_classes, growth_rate=12,
                 reduction=0.5, bottleneck=True, dropRate=0.0):
        super(DenseNet3, self).__init__()
        in_planes = 2 * growth_rate
        n = (depth - 4) / 3
        if bottleneck == True:
            n = n/2
            block = BottleneckBlock
        else:
            block = BasicBlock
        # 1st conv before any dense block
        self.conv1 = nn.Conv2d(3, in_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        self.trans1 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 2nd block
        self.block2 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        self.trans2 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 3rd block
        self.block3 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(in_planes, num_classes)
        self.in_planes = in_planes

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.block1(out))
        out = self.trans2(self.block2(out))
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 16)
        out = out.view(-1, self.in_planes)
        return self.fc(out)


In [11]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = 0.1 * (0.1 ** (epoch // 20))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


In [16]:
def train(train_loader, model, criterion, optimizer, epoch):
    """Train for one epoch on the training set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()
    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        adjust_learning_rate(optimizer, epoch)
        target_var = target.cuda()
        input_var = input.cuda()

        # compute output
        output = m(model(input_var))
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target_var, topk=(1,))[0]
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 5 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      loss=losses, top1=top1))
    # log to TensorBoard
        model.zero_grad()
        m.zero_grad()

def validate(val_loader, model, criterion, epoch):
    """Perform validation on the validation set"""
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        target_var = target.cuda()
        input_var = input.cuda()

        # compute output
        output = m(model(input_var))
     #   print (output)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target_var, topk=(1,))[0]
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

       # if i % 50 == 0:
       #     print('Test: [{0}/{1}]\t'
       #           'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
       #           'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
       #           'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
       #               i, len(val_loader), batch_time=batch_time, loss=losses,
       #               top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))
    # log to TensorBoard
  #  if args.tensorboard:
  #      log_value('val_loss', losses.avg, epoch)
  #      log_value('val_acc', top1.avg, epoch)
    return top1.avg

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.data.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res



In [17]:
import time
for j in range(0,50000):
    train(train_loader, model, criterion, optimizer, j)
    if j%1 == 0 :
        prec1 = validate(test_loader, model, criterion, j)
        torch.save(model.state_dict(), '/home/hubo/notebook/20170307/new_lesion_epoch_%d_%.1f.pth' % (j,prec1))
        torch.save(m.state_dict(), '/home/hubo/notebook/20170307/new_lesion_m_epoch_%d_%.1f.pth' % (j,prec1))

Epoch: [0][0/10]	Time 0.524 (0.524)	Loss 0.3778 (0.3778)	Prec@1 81.250 (81.250)
Epoch: [0][5/10]	Time 0.879 (0.823)	Loss 0.4899 (0.4475)	Prec@1 79.688 (80.990)
 * Prec@1 78.125
Epoch: [1][0/10]	Time 0.453 (0.453)	Loss 0.3515 (0.3515)	Prec@1 85.938 (85.938)
Epoch: [1][5/10]	Time 0.889 (0.817)	Loss 0.3968 (0.4137)	Prec@1 79.688 (81.771)
 * Prec@1 78.125
Epoch: [2][0/10]	Time 0.453 (0.453)	Loss 0.2862 (0.2862)	Prec@1 87.500 (87.500)
Epoch: [2][5/10]	Time 0.882 (0.816)	Loss 0.3606 (0.3762)	Prec@1 84.375 (83.073)
 * Prec@1 81.250
Epoch: [3][0/10]	Time 0.448 (0.448)	Loss 0.4047 (0.4047)	Prec@1 82.812 (82.812)
Epoch: [3][5/10]	Time 0.888 (0.819)	Loss 0.3595 (0.4219)	Prec@1 84.375 (82.812)
 * Prec@1 81.250
Epoch: [4][0/10]	Time 0.426 (0.426)	Loss 0.4704 (0.4704)	Prec@1 79.688 (79.688)
Epoch: [4][5/10]	Time 0.893 (0.814)	Loss 0.4068 (0.4298)	Prec@1 81.250 (81.510)
 * Prec@1 78.125
Epoch: [5][0/10]	Time 0.446 (0.446)	Loss 0.5369 (0.5369)	Prec@1 81.250 (81.250)
Epoch: [5][5/10]	Time 0.888 (0.816)

KeyboardInterrupt: 

In [ ]:
for i, (input, target) in enumerate(test_loader):
    target_var = target.cuda()
    input_var = input.cuda()

In [ ]:
import time

model.load_state_dict(torch.load('/home/hubo/notebook/densenet/params/1_epoch_100.pth'))
prec1 = validate(test_loader, model, criterion, 0)

In [ ]:
x1 = Variable(torch.randn(1,2))
x2 = Variable(torch.LongTensor(np.ones(1).astype(int)))
#loss = criterion(x1, x2)
loss = nn.CrossEntropyLoss()
a = loss(x1,x2)

In [ ]:
criterion()

In [ ]:
output
